In [2]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/the_wildfire_dataset_2n_version'
test_dir = '/content/drive/MyDrive/the_wildfire_dataset_2n_version1'

Mounted at /content/drive


In [12]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, ConcatDataset, Subset
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import timm

# # === Configuration ===
# train_dir = '../Project/the_wildfire_dataset_2n_version/train/'
# test_dir = '../Project/the_wildfire_dataset_2n_version/test/'
batch_size = 32
num_epochs = 10
img_size = 224
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# === Transforms ===
transform_train = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# === Dataset Loading ===
train_data = datasets.ImageFolder(os.path.join(train_dir, "train"), transform=transform_train)
test_data = datasets.ImageFolder(os.path.join(test_dir, "test"), transform=transform)
val_data = datasets.ImageFolder(os.path.join(test_dir, "val"), transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

class_names = train_data.classes

cuda


In [13]:
def build_model_from_scratch():
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    return model

def build_finetuned_model():
    model = models.resnet18(pretrained=True)  # Load pre-trained ResNet18
    for param in model.parameters():
        param.requires_grad = False  # Freeze all layers except the final fully connected layer

    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    return model

def ViT_model():
  model = timm.create_model('vit_base_patch16_224', pretrained=True)
  model.head = nn.Linear(model.head.in_features, len(class_names))
  model = model.to(device)
  return model

In [14]:
# def train_model(model, train_loader, val_loader, epochs=10):
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_val / total_val

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

In [15]:
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    # Metrics
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [7]:
# === Train from Scratch ===
print("Training model from scratch...")
model = build_model_from_scratch().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
evaluate_model(model, test_loader)

Training model from scratch...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10 | Train Loss: 0.7176, Train Acc: 0.6520 | Val Loss: 0.6236, Val Acc: 0.7189
Epoch 2/10 | Train Loss: 0.5625, Train Acc: 0.6970 | Val Loss: 0.6110, Val Acc: 0.7040
Epoch 3/10 | Train Loss: 0.5769, Train Acc: 0.6954 | Val Loss: 0.5297, Val Acc: 0.7139
Epoch 4/10 | Train Loss: 0.5326, Train Acc: 0.7230 | Val Loss: 0.5087, Val Acc: 0.7239
Epoch 5/10 | Train Loss: 0.5265, Train Acc: 0.7320 | Val Loss: 0.5972, Val Acc: 0.7363
Epoch 6/10 | Train Loss: 0.5019, Train Acc: 0.7542 | Val Loss: 0.5802, Val Acc: 0.6816
Epoch 7/10 | Train Loss: 0.4872, Train Acc: 0.7643 | Val Loss: 0.5804, Val Acc: 0.6940
Epoch 8/10 | Train Loss: 0.4805, Train Acc: 0.7744 | Val Loss: 0.4503, Val Acc: 0.7761
Epoch 9/10 | Train Loss: 0.4716, Train Acc: 0.7733 | Val Loss: 0.4932, Val Acc: 0.7711
Epoch 10/10 | Train Loss: 0.4668, Train Acc: 0.7823 | Val Loss: 0.5235, Val Acc: 0.7612


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Classification Report:
              precision    recall  f1-score   support

        fire       0.69      0.83      0.76       159
      nofire       0.88      0.77      0.82       251

    accuracy                           0.79       410
   macro avg       0.79      0.80      0.79       410
weighted avg       0.81      0.79      0.80       410

Confusion Matrix:
[[132  27]
 [ 58 193]]


In [8]:
# === Fine-tuning Pretrained Model ===
print("\nTraining model with fine-tuning...")
model = build_finetuned_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)  # Fine-tune only the last layer

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
evaluate_model(model, test_loader)


Training model with fine-tuning...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  w

Epoch 1/10 | Train Loss: 0.5781, Train Acc: 0.6986 | Val Loss: 0.5077, Val Acc: 0.7363
Epoch 2/10 | Train Loss: 0.4358, Train Acc: 0.8019 | Val Loss: 0.4150, Val Acc: 0.8085
Epoch 3/10 | Train Loss: 0.4020, Train Acc: 0.8141 | Val Loss: 0.3912, Val Acc: 0.8383
Epoch 4/10 | Train Loss: 0.3792, Train Acc: 0.8332 | Val Loss: 0.3801, Val Acc: 0.8458
Epoch 5/10 | Train Loss: 0.3562, Train Acc: 0.8475 | Val Loss: 0.3879, Val Acc: 0.8333
Epoch 6/10 | Train Loss: 0.3601, Train Acc: 0.8448 | Val Loss: 0.3844, Val Acc: 0.8159
Epoch 7/10 | Train Loss: 0.3223, Train Acc: 0.8713 | Val Loss: 0.3574, Val Acc: 0.8433
Epoch 8/10 | Train Loss: 0.3437, Train Acc: 0.8432 | Val Loss: 0.3784, Val Acc: 0.8383
Epoch 9/10 | Train Loss: 0.3394, Train Acc: 0.8586 | Val Loss: 0.3548, Val Acc: 0.8483
Epoch 10/10 | Train Loss: 0.3298, Train Acc: 0.8522 | Val Loss: 0.3739, Val Acc: 0.8333


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Classification Report:
              precision    recall  f1-score   support

        fire       0.91      0.78      0.84       159
      nofire       0.87      0.95      0.91       251

    accuracy                           0.89       410
   macro avg       0.89      0.87      0.88       410
weighted avg       0.89      0.89      0.88       410

Confusion Matrix:
[[124  35]
 [ 12 239]]


In [16]:
# === Fine-tuning Video Transformer ===
print("\nTraining model with fine-tuning...")
model= ViT_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(), lr=learning_rate)

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
evaluate_model(model, test_loader)


Training model with fine-tuning...


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10 | Train Loss: 0.3200, Train Acc: 0.8533 | Val Loss: 0.2324, Val Acc: 0.9129
Epoch 2/10 | Train Loss: 0.1992, Train Acc: 0.9237 | Val Loss: 0.2073, Val Acc: 0.9204
Epoch 3/10 | Train Loss: 0.1834, Train Acc: 0.9269 | Val Loss: 0.2228, Val Acc: 0.9104
Epoch 4/10 | Train Loss: 0.1531, Train Acc: 0.9439 | Val Loss: 0.2034, Val Acc: 0.9229
Epoch 5/10 | Train Loss: 0.1404, Train Acc: 0.9476 | Val Loss: 0.1784, Val Acc: 0.9254
Epoch 6/10 | Train Loss: 0.1422, Train Acc: 0.9449 | Val Loss: 0.1726, Val Acc: 0.9328
Epoch 7/10 | Train Loss: 0.1266, Train Acc: 0.9497 | Val Loss: 0.1786, Val Acc: 0.9328
Epoch 8/10 | Train Loss: 0.1221, Train Acc: 0.9507 | Val Loss: 0.1780, Val Acc: 0.9279
Epoch 9/10 | Train Loss: 0.1209, Train Acc: 0.9534 | Val Loss: 0.1707, Val Acc: 0.9328
Epoch 10/10 | Train Loss: 0.1183, Train Acc: 0.9566 | Val Loss: 0.1761, Val Acc: 0.9303


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Classification Report:
              precision    recall  f1-score   support

        fire       0.93      0.94      0.93       159
      nofire       0.96      0.95      0.96       251

    accuracy                           0.95       410
   macro avg       0.94      0.94      0.94       410
weighted avg       0.95      0.95      0.95       410

Confusion Matrix:
[[149  10]
 [ 12 239]]
